In [17]:
import numpy as np
import time
from scipy.optimize import ridder

def timeit(func):
    def wrapper(*arg, **kw):
        t1 = time.perf_counter()
        res = func(*arg, **kw)
        t2 = time.perf_counter()
        print("{0} took {1}s".format(func.__name__, (t2 - t1)))
        return res
    return wrapper

arr = np.genfromtxt("xdata.txt")

mu_hat = arr.mean()


def funderivativel(lambdas, mu, arr):
    n = len(arr)
    log_likelihoods_derivatives = (n/(2*lambdas)) - (((arr - mu)**2)/((2*mu**2)*arr)).sum()
    return log_likelihoods_derivatives

In [18]:
@timeit
def rootfindscipy(f, a, b, args):
    return ridder(f, a, b,  args)

rootfindscipy(funderivativel, 0.1, 3, (mu_hat, arr))

rootfindscipy took 0.0005708999997295905s


1.477439027305124

In [19]:
@timeit
def bisection(f, a, b, args, N_MAX, DELTA):
    if f(a, *args)*f(b, *args) > 0:
        raise Exception("f(a)f(b) > 0, This function may not converge")

    n = 0
    while n < N_MAX:
        c = (b + a) / 2
        if f(c, *args) == 0 or c < DELTA:
            return c
        n += 1

        if f(c, *args)*f(a, *args) > 0:
            a = c
        else:
            b = c
        
    raise Exception("No solution could be found")

bisection(funderivativel, 0.1, 3, (mu_hat, arr), 100, 0.001)

bisection took 0.005296199997246731s


1.4774390273041236